In [1]:
%%html
<link rel="stylesheet" href="static/hyrule.css" type="text/css">

In [2]:
# Imports numpy as np and matplotlib.pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection as f_select
from sklearn import cross_validation as cv
from __future__ import division
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('ggplot')

In [4]:
ames = pd.read_csv("../data/ames_train.csv", header=0, na_values="?")

In [5]:
ames.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
print(ames.shape)

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

ames_numerics = ames.select_dtypes(include=numerics)
print(ames_numerics.shape)

# Below we are dropping features that have NA values. 
# It may be worth filling in the values somehow, as this solution loses 23% of the features.

ames_numerics = ames_numerics.dropna()
print(ames_numerics.shape)

(1460, 81)
(1460, 38)
(1121, 38)


In [7]:
x_columns = list(ames_numerics.columns)
y_column = "SalePrice"
x_columns.remove(y_column)

# The goal of this for loop is to establish statistically significant columns and only work with those.
# Note: I had to run .dropna() on ames_numerics (implemented several cells above) to get this for loop to work.
significant_columns = []
nonsignificant_columns = []
pvals = []
nonsignificant_pvals = []
for feature in x_columns:
    pval = f_select.f_regression(ames_numerics[[feature]], ames_numerics[y_column])
    if pval[1][0] < 0.05:
        significant_columns.append(feature)
        pvals.append(pval[1][0])
    else:
        nonsignificant_columns.append(feature)
        nonsignificant_pvals.append(pval[1][0])

print(significant_columns)
print(pvals)

x_train, x_test, y_train, y_test = cv.train_test_split(ames_numerics[significant_columns],
                                                           ames_numerics[y_column],
                                                           test_size=0.333,
                                                           random_state=1234)

model = LinearRegression().fit(x_train, y_train)

print(pd.DataFrame({
    'column': significant_columns,
    'coef': model.coef_,
    'p-value': pvals,
}).set_index('column').sort('p-value'))

print(model.score(x_train, y_train))
print(metrics.r2_score(y_train, model.predict(x_train)))
print(model.score(x_test, y_test))
print(metrics.r2_score(y_test, model.predict(x_test)))

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'PoolArea']
[0.0031791862081776154, 1.5261773976090531e-32, 9.6881587877969871e-25, 3.1736090867026826e-248, 2.9619968293510947e-05, 1.4681206422179375e-80, 4.1716366544814187e-79, 2.454594130032088e-68, 4.2219371830390603e-42, 5.5592652066787747e-13, 6.7714215730174999e-118, 2.9374157361826968e-114, 7.1313694102218095e-26, 2.7499470443203481e-169, 9.5987475151416791e-16, 3.3931133682049853e-96, 5.6753557417380467e-20, 1.9281038132888889e-08, 2.3326641675096397e-06, 1.6584601808570785e-88, 2.5618863527163848e-60, 1.6513295339131269e-73, 5.8587196428268202e-134, 1.0596837938035933e-119, 3.811628

/Users/Darin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


NameError: name 'metrics' is not defined